# Send request to filter example

To use this example, start a Diffusion server and update the connection details below as needed.

The example creates two sessions, sets a request handler on one of them (the receiver) and the filter response handler on the other (the sender). The sender then sends a message to the filter, and waits to receive a response from the receiver.

In [ ]:
import asyncio
import diffusion

In [ ]:
# Diffusion server connection information; same for both sessions
# adjust as needed for the server used in practice
server_url = "ws://localhost:8080"
principal = "admin"
credentials = diffusion.Credentials("password")

# request properties
request = "Pushme Pullyou"
path = "path"
request_type = diffusion.datatypes.STRING  # datatype of the request

In [ ]:
# request handler
def callback(request: str, **kwargs) -> str:
    return f"Hello there, {request}!"

request_handler = diffusion.messaging.RequestHandler(
    callback, request_type=request_type, response_type=request_type
)

# filter response handler
def filter_response_callback(response, **kwargs):
    print("Received response from session '{session_id}':".format(**kwargs))
    print(f"    {response}")
    print(" - Request was sent to {filter} on path {path}".format(**kwargs))
    print(" - Received {received} of {expected} response(s).".format(**kwargs))

filter_response_handler = diffusion.handlers.EventStreamHandler(
    response=filter_response_callback
)

In [ ]:
# creating the two sessions
sender = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)
receiver = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)

async with sender, receiver:

    # adding the handler for addressed requests
    receiver.messaging.add_stream_handler(path, handler=request_handler, addressed=True)

    # specifying the session filter
    # this is a very simple filter, addressing the receiver session directly
    session_filter = f"$SessionId is '{receiver.session_id}'"

    # adding filter response handler
    sender.messaging.add_filter_response_handler(
        session_filter=session_filter, handler=filter_response_handler
    )

    # sending the request and receiving the number of expected responses
    print(f"Sending request: '{request}' to session filter `{session_filter}`...")
    try:
        response = await sender.messaging.send_request_to_filter(
            session_filter=session_filter, path=path, request=request_type(request),
        )
    except diffusion.DiffusionError as ex:
        print(f"ERROR: {ex}")
    else:
        print(f"... expecting {response} response(s) ...")

    # waiting a bit to receive all responses
    await asyncio.sleep(1)
